# OLS Baseline 
In this notebook an OLS estimation will be performed which serves as a baseline to compare the other methods with. It also serves as the proof-of-concept to figure out how to structure the data correctly and save results before moving on to other architectures.

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
def createRollingWindow(dataset, look_back = 1):
    """
    Function takes a 2 dimensional array as input and outputs a 2 dimensional array containing rolling windows of the matrix of size [No_Obs - look_back, look_back * No_Vars].
    It creates rolling windows through concatenating all variables at time t with all variables at time t+1 etc until you you have reached t+look_back and move to next window. 
    """
    X= pd.DataFrame(np.empty((dataset.shape[0] - look_back, dataset.shape[1] * look_back)))
    for i in tqdm(range(dataset.shape[0] - look_back)):    
        X.iloc[i] = dataset.iloc[i:(i + look_back):].to_numpy().flatten()
    return X

In [4]:
def createRollingWindow1D(dataset, look_back = 1):
    """
    Function takes a 1 dimensional array as input and outputs a 2 dimensional array containing rolling windows of the series of size look_back.
    """
    X= pd.DataFrame(np.empty((dataset.shape[0] - look_back, look_back)))
    for i in tqdm(range(dataset.shape[0] - look_back)):    
        X.iloc[i] = dataset.iloc[i:(i + look_back):].to_numpy().flatten()
    return X

In [5]:
def shift_data(steps, X, y):
    """
    Function takes a 2D X matrix and 1d y series and shifts them with steps and resizes them such that there are no empty lines. 
    """
    X = X[:X.shape[0]-steps]
    y = y.shift(periods=-steps)[:y.shape[0]-steps].reset_index(drop=True)
    return X,y

## Reading Data
First we start with loading the relevant data from the excel to be used in our analyis

In [103]:
#Read the equity premium series to a dataframe
ep = pd.read_excel('data/Augemented_Formatted_results.xls', sheet_name='Equity premium', skiprows= range(1118,1127,1))[:-1]
ep['Date'] = pd.to_datetime(ep['Date'], format='%Y%m')
ep = ep.set_index('Date')
ep = ep.loc[(ep.index >= '1950-12-01')]

In [104]:
#Read the maacroeconomic variables to a dataframe
mev = pd.read_excel('data/Augemented_Formatted_results.xls', sheet_name='Macroeconomic variables', 
                    skiprows= range(1118,1126,1)).fillna(method='bfill')[:-1] #backward fill missing values. 
mev = mev.loc[:, ~mev.columns.str.match('Unnamed')]  #Remove empty column
mev['Date'] = pd.to_datetime(mev['Date'], format='%Y%m') #convert date pandas format
mev = mev.set_index('Date') #Set date as index. 
mev = mev.loc[(mev.index >= '1950-12-01')]

In [105]:
ta = pd.read_excel('data/Augemented_Formatted_results.xls', sheet_name='Technical indicators', 
                    skiprows= range(1118,1119,1))[:-1]
ta['Date'] = pd.to_datetime(ta['Date'], format='%Y%m')
ta = ta.set_index('Date')
ta = ta.loc[(ta.index >= '1950-12-01')]

### Data restructuring
We must create rolling windows of the Macro Economic Variables (MEV) and match them with the 1 month out of sample equity premium in order train a model. 

In [9]:
#Create rolling window version of the MEV dataset.  
X_mev = createRollingWindow(mev, look_back = 12)

In [10]:
#Shift equity premiumms such that they correspond to the 1 month out of sample corresponding to each window. 
y = ep.shift(periods=-12)[:ep.shape[0]-12].reset_index(drop=True)

#Convert y to a series with only log equity premium or simple equity premium 
y = y['Log equity premium'].astype('float64')

### Train OLS Model Windowed
Create rolling windows where we try to predit the 1 month out of sample equity premium based on the previous 12 months of Macro economic variables.

In [11]:
#Create Train and test set
X_train, X_test, y_train, y_test = train_test_split(X_mev, y, train_size=168, random_state=0, shuffle=False)

In [12]:
#Train a linear regression model on MEV rolling window data and the corresponding 1 month out of sample equity premium. 
reg = LinearRegression().fit(X_train, y_train)
coefficients = reg.coef_
intercept = reg.intercept_

In [13]:
#Make a prediction
y_pred = reg.predict(X_test)

from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2:', metrics.r2_score(y_test, y_pred))
print('Explained Variance:', metrics.explained_variance_score(y_test, y_pred))

Mean Absolute Error: 15.805841849587061
Mean Squared Error: 977.3759656738616
Root Mean Squared Error: 31.263012741478732
R2: -525094.7996636973
Explained Variance: -488032.71709039697


### Train OLS Model Vanilla
Train an OLS model without the rolling window variation. Here we just shift the equity premium by 1 such that we alling 1 row of MEV measurements with the 1 month out of sample equity premium. Thus here an OLS is trained on 1 month of MEV variables with the 1 month out of sample equity risk premium. We are NOT looking at the last 12 months in this example. 

In [14]:
X = mev[:mev.shape[0]-1]
y = ep['Log equity premium'].shift(periods=-1)[:ep['Log equity premium'].shape[0]-1].reset_index(drop=True)

In [15]:
#Create Train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=168, random_state=0, shuffle=False)

In [16]:
#Train a linear regression model on MEV rolling window data and the corresponding 1 month out of sample equity premium. 
reg = LinearRegression().fit(X_train, y_train)
coefficients = reg.coef_
intercept = reg.intercept_

In [17]:
#Make a prediction
y_pred = reg.predict(X_test)

from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2:', metrics.r2_score(y_test, y_pred))
print('Explained Variance:', metrics.explained_variance_score(y_test, y_pred))

Mean Absolute Error: 0.40171447999422094
Mean Squared Error: 0.2844314592484045
Root Mean Squared Error: 0.533321159573108
R2: -153.54061176469264
Explained Variance: -66.74475794944357


## Rolling window OLS estimation for each variable separately
Up till now I have combined all variables into 1 window, however this is not in line with Neely, Rapach, Tu and Zhou (2014). They run seperate regression for each macro economic variable and TA variable, and hence that is what should also do. 

rollingWindowMEV is a dictionary which will be filled with the 2D dataframes containing the rolling windows for a single variable. Thus rollingWindowsMEV['DP'] would yield the 2D matrix containing the rolling windows for the variable DP and will be of size [No_obs-window_size, window_size] or in our case [828 obs - 12, 12] = [817,12]. The dictionairy is thus accessible with the variable name as index key and these can be obtained from the original data through mev.columns which yields an array of the column names of the original dataframe.

In [106]:
#Shift equity premiumms such that they correspond to the 1 month out of sample corresponding to each window. 
y = ep.shift(periods=-12)[:ep.shape[0]-12].reset_index(drop=True)

#Convert y to a series with only log equity premium or simple equity premium 
y = y['Log equity premium'].astype('float64')

In [107]:
# Create empty dictionary
rollingWindowsMEV = dict()

#Fill the dictionairy with the 2D array with rolling windows for each variable. 
for variable in mev.columns:
    rollingWindowsMEV[variable] = createRollingWindow1D(mev[variable], 12)

In [110]:
df = pd.DataFrame(columns=['Variable', 'Coef', 'Intercept', 'R2'])

for variable in mev.columns:
    X_train, X_test, y_train, y_test = train_test_split(rollingWindowsMEV[variable], y, train_size=168, random_state=0, shuffle=False)
    reg = LinearRegression().fit(X_train, y_train)
    
    df = df.append(pd.Series({'Variable' : variable, 
                              'Coef' : reg.coef_[0], 
                              'Intercept' : reg.intercept_, 
                              'R2':  reg.score(X_train, y_train)}), ignore_index=True)
    
    y_pred = reg.predict(X_test)
#     print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
#     print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
#     print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
#     print('R2:', metrics.r2_score(y_test, y_pred))
#     print('Explained Variance:', metrics.explained_variance_score(y_test, y_pred))
#     print('\n')

In [111]:
df

,Variable,Coef,Intercept,R2
0,DP,-0.008935,0.064910,0.072770
1,DY,0.001962,0.068221,0.075063
2,EP,0.014268,0.049222,0.067791
3,DE,-0.030262,0.005276,0.053072
4,RVOL,-0.107951,0.008718,0.079077
5,BM,0.051665,-0.027054,0.082900
6,NTIS,-0.536463,0.023586,0.073837
7,TBL (ann %),-0.007852,0.027838,0.140212
8,LTY (ann %),-0.007834,0.039370,0.095641
9,LTR (%),0.004386,0.005874,0.100159


# Comparisson to Rapach
This is the exact in-sample predicitve regression as run in the Rapach paper of which the results can be found in table 2. This is just to confirm that the findings are in order. 
It is the following bi-variate regression which is run on the data from 1951:01 - 2011:12.  
$$ r_{t+1} = \alpha_i +\beta_i q_{i,t} + \epsilon_{i,t+1}$$


In [99]:
#Shift equity premiumms such that they correspond to the 1 month out of sample corresponding to each window. 
y = ep.shift(periods=-1)[:ep.loc[(ep.index <= '2011-12-01')].shape[0]-1].reset_index(drop=True)

#Convert y to a series with only log equity premium or simple equity premium 
y = y['Log equity premium'].astype('float64')

# Remove the last observation such that the size of the dataamtrix coincides with the shifted y euity ridk premium
X = mev[:mev.loc[(mev.index <= '2011-12-01')].shape[0]-1]

In [100]:
df = pd.DataFrame(columns=['Variable', 'Coef', 'Intercept', 'R2'])
for variable in mev.columns:
#     X_train, X_test, y_train, y_test = train_test_split(, y, train_size=168, random_state=0, shuffle=False)
    reg = LinearRegression().fit(X[variable].values.reshape(-1, 1), y)
    df = df.append(pd.Series({'Variable' : variable, 
                              'Coef' : reg.coef_[0], 
                              'Intercept' : reg.intercept_, 
                              'R2':  reg.score(X[variable].values.reshape(-1,1), y)}), ignore_index=True)

    

In [101]:
df

,Variable,Coef,Intercept,R2
0,DP,0.007794,0.031787,0.005802
1,DY,0.008356,0.033700,0.006702
2,EP,0.004349,0.016671,0.001982
3,DE,0.005918,0.008803,0.001708
4,RVOL,0.074078,-0.006035,0.007336
5,BM,0.005359,0.001713,0.000988
6,NTIS,-0.006571,0.004691,0.000008
7,TBL (ann %),-0.001084,0.009667,0.005648
8,LTY (ann %),-0.000756,0.009371,0.002256
9,LTR (%),0.001345,0.003850,0.007570
